<a href="https://colab.research.google.com/github/SuminBae97/PytorchStudy/blob/main/gluon_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mxnet


In [ ]:
pip install --upgrade mxnet~=1.7 gluonts

In [ ]:
pip install d2l

In [ ]:
pip install -U d2l

In [65]:
import sys
import d2l
from mxnet.gluon import data as gdata
import sys,time
from mxnet import autograd,nd
from d2l import mxnet as d2l

In [30]:
import sys
sys.path.insert(0, '..')

%matplotlib inline
import d2l
from mxnet.gluon import data as gdata
import sys
import time

In [6]:
mnist_train = gdata.vision.FashionMNIST(train=True)
mnist_test = gdata.vision.FashionMNIST(train=False)

In [25]:
mnist_train[0][0].shape

(28, 28, 1)

In [26]:
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]
def show_fashion_mnist(images, labels):
    #d2l.use_svg_display()
    # Here _ means that we ignore (not use) variables
    _, figs = d2l.plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.reshape((28, 28)).asnumpy(),cmap='grey')
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)

In [31]:

batch_size = 256
transformer = gdata.vision.transforms.ToTensor()
if sys.platform.startswith('win'):
    # 0 means no additional processes are needed to speed up the reading of
    # data
    num_workers = 0
else:
    num_workers = 4

train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                              batch_size, shuffle=True,
                              num_workers=num_workers)
test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                             batch_size, shuffle=False,
                             num_workers=num_workers)

In [33]:
train_iter

In [34]:
num_inputs=784
num_outputs=10
W = nd.random.normal(scale=0.01,shape=(num_inputs,num_outputs))
b= nd.zeros(num_outputs)


In [35]:
W.attach_grad()
b.attach_grad()

In [36]:
test = nd.array([[1,2,3]])

In [55]:
def softmax(X):
    X_exp = X.exp()
    partition = X_exp.sum(axis=1,keepdims=True)
    return X_exp/partition

def net(X):
    return softmax(nd.dot(X.reshape((-1,num_inputs)),W)+b)

def cross_entropy(y_hat,y):
    return -nd.pick(y_hat,y).log()

def accuracy(y_hat,y):
    return (y_hat.argmax(axis=1)==y.astype('float32')).mean().asscalar()

In [41]:
X=nd.random.normal(shape=(2,5))
X_prob = softmax(X)


In [43]:
X_prob,X_prob.sum(axis=1)

(
 [[0.21324193 0.33961776 0.1239742  0.27106097 0.05210521]
  [0.11462264 0.3461234  0.19401033 0.29583326 0.04941036]]
 <NDArray 2x5 @cpu(0)>, 
 [1.0000001 1.       ]
 <NDArray 2 @cpu(0)>)

In [61]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        y = y.astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum().asscalar()
        n += y.size
    return acc_sum / n

In [71]:
num_epochs, lr = 5, 0.1

# This function has been saved in the d2l package for future use
def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, trainer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_iter:
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            if trainer is None:
                d2l.sgd(params, lr, batch_size)
            else:
                # This will be illustrated in the next section
                trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

train_ch3(net, train_iter, test_iter, cross_entropy, 20,
          batch_size, [W, b], lr)

epoch 1, loss 0.4777, train acc 0.837, test acc 0.844
epoch 2, loss 0.4693, train acc 0.840, test acc 0.845
epoch 3, loss 0.4624, train acc 0.841, test acc 0.847
epoch 4, loss 0.4559, train acc 0.846, test acc 0.847
epoch 5, loss 0.4509, train acc 0.847, test acc 0.847
epoch 6, loss 0.4465, train acc 0.848, test acc 0.850
epoch 7, loss 0.4428, train acc 0.848, test acc 0.851
epoch 8, loss 0.4404, train acc 0.850, test acc 0.852
epoch 9, loss 0.4371, train acc 0.851, test acc 0.852
epoch 10, loss 0.4345, train acc 0.851, test acc 0.851
epoch 11, loss 0.4319, train acc 0.852, test acc 0.851
epoch 12, loss 0.4294, train acc 0.853, test acc 0.849
epoch 13, loss 0.4275, train acc 0.853, test acc 0.855
epoch 14, loss 0.4257, train acc 0.854, test acc 0.855
epoch 15, loss 0.4227, train acc 0.856, test acc 0.854
epoch 16, loss 0.4217, train acc 0.855, test acc 0.853
epoch 17, loss 0.4207, train acc 0.856, test acc 0.853
epoch 18, loss 0.4199, train acc 0.855, test acc 0.851
epoch 19, loss 0.41